In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
import scipy.sparse as sp

In [2]:
N=6   #time       
M=3   #plant
K=5   #item
L=6   #l
E=3
A=2   #a

H=np.array([[2,1,3]])
H=np.tile(H,(5,1))

I=np.eye(N)

D=np.array([[15,20,20,25,28,22],[0,0,0,0,0,0],[7,12,15,8,13,16],[21,10,17,16,23,25],[0,0,0,0,0,0]])
DD=np.zeros((K,N))
h,w=D.shape
for i in range(h):
    for j in range(w):
        if D[i][j]!=0:
            DD[i][j]=1/D[i][j]
            
epsilonUD=np.ones(N)*0.05
epsilonUI=0.8

v0=np.array([[20,30,20]]) 
v0=np.tile(v0,(K,1))

nu=np.array([[2],[1],[1],[1],[1]])
nu=np.tile(nu,(1,M))

C=np.array([[30],[15],[20]])
C=np.tile(nu,(1,N))

cL=np.array([[1,2],[2,1],[1,3],[3,1],[2,3],[3,2]])
cE=np.array([[2,1],[3,1],[5,4]])
cA=np.array([[2,5],[5,2]])

dtUL=np.array([2,2,2,2,1,1])  #delta t^L _il
dtUP=2 #delta t^P_ij

Q=np.zeros((K,M,N))
q=np.array([3,3,2])

m=5  #m_i
wLB=1
wUB=20

RC=np.array([[0,0.1,0],[0,0,0.1]])

In [3]:
prob=gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-15


In [4]:
#variable
v=prob.addMVar((K,M,N))    #(i,j,t)
u=prob.addMVar((K,N))   #(i,t)
z=prob.addMVar((K,M,N))  #(i,j,t)
yUI=prob.addMVar((K,M,N))    #(i,j,t)
yUO=prob.addMVar((K,M,N))    #(i,j,t)
x=prob.addMVar((K,M,N))    #(i,j,t)
r=prob.addMVar((A,M,N))    #(a,j,t)
s=prob.addMVar((K,L,N))    #(i,l,t)
w=prob.addMVar((K,M,N),vtype=GRB.INTEGER)    #(i,j,t)
xUb=prob.addMVar((K,M,N),vtype=GRB.BINARY)    #(i,j,t)

In [5]:
#constraint()
#(1b)-(1c)
prob.addConstrs(u[i][0]+np.ones((1,M))@z[i][:,0]==D[i][0] for i in range(K))
prob.addConstrs(u[i][t]-u[i][t-1]+np.ones((1,M))@z[i][:,t]==D[i][t] for i in range(K)
                                                                    for t in range(1,N))

#(1d)
te=0
for t in range(N):
    for i in range(K):
        te+=u[i][t]*DD[i][t]
    prob.addConstr(te<=epsilonUD[t])

#(1e)
prob.addConstrs(v[i][j][0]-yUI[i][j][0]+yUO[i][j][0]==v0[i][j] for i in range(K)
                                                              for j in range(M))
prob.addConstrs(v[i][j][t]-v[i][j][t-1]-yUI[i][j][t]+yUO[i][j][t]==0 for i in range(K)
                                                                     for j in range(M)
                                                                     for t in range(1,N))

#(1f)
for i in range(K):
    te=0
    for t in range(N):
        for j in range(M):
            te+=360/(12*N)*v[i][j][t]-epsilonUI*yUO[i][j][t]
        prob.addConstr(te<=0)

#(1g)
for i in range(K):
    for j in range(M):
        for t in range(N):
            te=0
            for l in range(L):
                if cL[l][1]==j+1 and t-dtUL[l]>=0:
                    te+=s[i][l][t-dtUL[l]]
            if t-dtUP>=0:
                te+=x[i][j][t-dtUP]+Q[i][j][t]
            te+=Q[i][j][t]
            prob.addConstr(yUI[i][j][t]-te==0)

#(1h)
for i in range(K):
    for j in range(M):
        for t in range(N):
            te=0
            for l in range(L):
                if cL[l][0]==j+1:
                    te+=s[i][l][t]
            for e in range(E):
                if cE[e][0]==i+1:
                    te+=q[e]*x[cE[e][1]-1][j][t]
            te+=z[i][j][t]
            for a in range(A):
                if cA[a][0]==i+1:
                    te+=r[a][j][t]
                if cA[a][1]==i+1:
                    te-=r[a][j][t]
            prob.addConstr(yUO[i][j][t]-te==0)

#(1i)
for j in range(M):
    for t in range(N):
        te=0
        for i in range(K):
            te+=nu[i][j]*x[i][j][t]
        prob.addConstr(te<=C[j][t])

#(1j)
for i in range(K):
    for a in range(A):
        if cA[a][0]==i+1:
            for j in range(M):
                for t in range(N):
                    prob.addConstr(r[a][j][t]>=0)
                    prob.addConstr(v[i][j][t]-r[a][j][t]>=0)

#(1k)-(1n)
for i in range(K):
    for t in range(N):
        for j in range(M):
            prob.addConstr(yUO[i][j][t]>=0)
            prob.addConstr(yUO[i][j][t]-v[i][j][t]<=0)
            prob.addConstr(x[i][j][t]>=0)
            prob.addConstr(yUI[i][j][t]>=0)
            prob.addConstr(v[i][j][t]>=0)
            prob.addConstr(z[i][j][t]>=0)
        prob.addConstr(u[i][t]>=0)
        for l in range(L):
            prob.addConstr(s[i][l][t]>=0)

#(1o)
prob.addConstrs(x[i][j][t]-m*w[i][j][t]==0 for i in range(K)
                                           for j in range(M)
                                           for t in range(N))

#(1p)
for i in range(K):
    for j in range(M):
        for t in range(N):
            prob.addConstr(xUb[i][j][t]*wLB-w[i][j][t]<=0)
            prob.addConstr(xUb[i][j][t]*wUB-w[i][j][t]>=0)

#(1q)
prob.addConstrs(w[i][j][t]>=0 for i in range(K)
                              for j in range(M)
                              for t in range(N))

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Constr *Awaiting Model Upd

In [6]:
#objective
f=0
for i in range(K):
    for j in range(M):
        f+=H[i][j]*(v[i][j].sum())
for a in range(A):
    for j in range(M):
        f+=RC[a][j]*(r[a][j].sum())
prob.setObjective(f) 

In [7]:
prob.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1536 rows, 876 columns and 3617 nonzeros
Model fingerprint: 0x907e1482
Variable types: 696 continuous, 180 integer (90 binary)
Coefficient statistics:
  Matrix range     [4e-02, 2e+01]
  Objective range  [1e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 3e+01]
Presolve removed 1023 rows and 323 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
